In [9]:
from transformers import pipeline
from secret import HUGGINGFACE_TOKEN, OPENAI_API_KEY
import torch

pipe = pipeline(
    "text-generation",
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    token=HUGGINGFACE_TOKEN,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda",
    max_new_tokens=12_000,
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
type(pipe.model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [2]:
i = ["Nice to see that one", "It's interesting."]

In [8]:
pipe.tokenizer.eos_token

# output_hidden_states

'<|eot_id|>'

In [7]:
pipe.tokenizer.pad_token_id

In [6]:
pipe.tokenizer.pad_token

In [4]:
pipe.tokenizer(i, return_tensors='pt',padding=True)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [2]:
# {'role': 'user', 'content': }
messages = [
    # {
    #     "role": "system",
    #     "content": "You are an expert in extracting entities and relationships from text. Identify key entities and their relationships clearly.",
    # },
    {
        "role": "user",
        "content": '\n-Goal-\nGiven a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.\n\n-Steps-\n1. Identify all entities. For each identified entity, extract the following information:\n- entity_name: Name of the entity, capitalized\n- entity_type: One of the following types: [disease,symptom,cause]\n- entity_description: Comprehensive description of the entity\'s attributes and activities\nFormat each entity as ("entity"<|><entity_name><|><entity_type><|><entity_description>\n\n2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.\nFor each pair of related entities, extract the following information:\n- source_entity: name of the source entity, as identified in step 1\n- target_entity: name of the target entity, as identified in step 1\n- relationship_description: explanation as to why you think the source entity and the target entity are related to each other\n- relationship_strength: a numeric score indicating strength of the relationship between the source entity and target entity\n Format each relationship as ("relationship"<|><source_entity><|><target_entity><|><relationship_description><|><relationship_strength>)\n\n3. Return output in English as a single list of all the entities and relationships identified in steps 1 and 2. Use **##** as the list delimiter.\n\n4. When finished, output <|COMPLETE|>\n\n######################\n-Examples-\n######################\nExample 1:\n\nEntity_types: [person, technology, mission, organization, location]\nText:\nwhile Alex clenched his jaw, the buzz of frustration dull against the backdrop of Taylor\'s authoritarian certainty. It was this competitive undercurrent that kept him alert, the sense that his and Jordan\'s shared commitment to discovery was an unspoken rebellion against Cruz\'s narrowing vision of control and order.\n\nThen Taylor did something unexpected. They paused beside Jordan and, for a moment, observed the device with something akin to reverence. “If this tech can be understood..." Taylor said, their voice quieter, "It could change the game for us. For all of us.”\n\nThe underlying dismissal earlier seemed to falter, replaced by a glimpse of reluctant respect for the gravity of what lay in their hands. Jordan looked up, and for a fleeting heartbeat, their eyes locked with Taylor\'s, a wordless clash of wills softening into an uneasy truce.\n\nIt was a small transformation, barely perceptible, but one that Alex noted with an inward nod. They had all been brought here by different paths\n################\nOutput:\n("entity"<|>"Alex"<|>"person"<|>"Alex is a character who experiences frustration and is observant of the dynamics among other characters.")##\n("entity"<|>"Taylor"<|>"person"<|>"Taylor is portrayed with authoritarian certainty and shows a moment of reverence towards a device, indicating a change in perspective.")##\n("entity"<|>"Jordan"<|>"person"<|>"Jordan shares a commitment to discovery and has a significant interaction with Taylor regarding a device.")##\n("entity"<|>"Cruz"<|>"person"<|>"Cruz is associated with a vision of control and order, influencing the dynamics among other characters.")##\n("entity"<|>"The Device"<|>"technology"<|>"The Device is central to the story, with potential game-changing implications, and is revered by Taylor.")##\n("relationship"<|>"Alex"<|>"Taylor"<|>"Alex is affected by Taylor\'s authoritarian certainty and observes changes in Taylor\'s attitude towards the device."<|>7)##\n("relationship"<|>"Alex"<|>"Jordan"<|>"Alex and Jordan share a commitment to discovery, which contrasts with Cruz\'s vision."<|>6)##\n("relationship"<|>"Taylor"<|>"Jordan"<|>"Taylor and Jordan interact directly regarding the device, leading to a moment of mutual respect and an uneasy truce."<|>8)##\n("relationship"<|>"Jordan"<|>"Cruz"<|>"Jordan\'s commitment to discovery is in rebellion against Cruz\'s vision of control and order."<|>5)##\n("relationship"<|>"Taylor"<|>"The Device"<|>"Taylor shows reverence towards the device, indicating its importance and potential impact."<|>9)<|COMPLETE|>\n#############################\nExample 2:\n\nEntity_types: [person, technology, mission, organization, location]\nText:\nThey were no longer mere operatives; they had become guardians of a threshold, keepers of a message from a realm beyond stars and stripes. This elevation in their mission could not be shackled by regulations and established protocols—it demanded a new perspective, a new resolve.\n\nTension threaded through the dialogue of beeps and static as communications with Washington buzzed in the background. The team stood, a portentous air enveloping them. It was clear that the decisions they made in the ensuing hours could redefine humanity\'s place in the cosmos or condemn them to ignorance and potential peril.\n\nTheir connection to the stars solidified, the group moved to address the crystallizing warning, shifting from passive recipients to active participants. Mercer\'s latter instincts gained precedence— the team\'s mandate had evolved, no longer solely to observe and report but to interact and prepare. A metamorphosis had begun, and Operation: Dulce hummed with the newfound frequency of their daring, a tone set not by the earthly\n#############\nOutput:\n("entity"<|>"Washington"<|>"location"<|>"Washington is a location where communications are being received, indicating its importance in the decision-making process.")##\n("entity"<|>"Operation: Dulce"<|>"mission"<|>"Operation: Dulce is described as a mission that has evolved to interact and prepare, indicating a significant shift in objectives and activities.")##\n("entity"<|>"The team"<|>"organization"<|>"The team is portrayed as a group of individuals who have transitioned from passive observers to active participants in a mission, showing a dynamic change in their role.")##\n("relationship"<|>"The team"<|>"Washington"<|>"The team receives communications from Washington, which influences their decision-making process."<|>7)##\n("relationship"<|>"The team"<|>"Operation: Dulce"<|>"The team is directly involved in Operation: Dulce, executing its evolved objectives and activities."<|>9)<|COMPLETE|>\n#############################\nExample 3:\n\nEntity_types: [person, role, technology, organization, event, location, concept]\nText:\ntheir voice slicing through the buzz of activity. "Control may be an illusion when facing an intelligence that literally writes its own rules," they stated stoically, casting a watchful eye over the flurry of data.\n\n"It\'s like it\'s learning to communicate," offered Sam Rivera from a nearby interface, their youthful energy boding a mix of awe and anxiety. "This gives talking to strangers\' a whole new meaning."\n\nAlex surveyed his team—each face a study in concentration, determination, and not a small measure of trepidation. "This might well be our first contact," he acknowledged, "And we need to be ready for whatever answers back."\n\nTogether, they stood on the edge of the unknown, forging humanity\'s response to a message from the heavens. The ensuing silence was palpable—a collective introspection about their role in this grand cosmic play, one that could rewrite human history.\n\nThe encrypted dialogue continued to unfold, its intricate patterns showing an almost uncanny anticipation\n#############\nOutput:\n("entity"<|>"Sam Rivera"<|>"person"<|>"Sam Rivera is a member of a team working on communicating with an unknown intelligence, showing a mix of awe and anxiety.")##\n("entity"<|>"Alex"<|>"person"<|>"Alex is the leader of a team attempting first contact with an unknown intelligence, acknowledging the significance of their task.")##\n("entity"<|>"Control"<|>"concept"<|>"Control refers to the ability to manage or govern, which is challenged by an intelligence that writes its own rules.")##\n("entity"<|>"Intelligence"<|>"concept"<|>"Intelligence here refers to an unknown entity capable of writing its own rules and learning to communicate.")##\n("entity"<|>"First Contact"<|>"event"<|>"First Contact is the potential initial communication between humanity and an unknown intelligence.")##\n("entity"<|>"Humanity\'s Response"<|>"event"<|>"Humanity\'s Response is the collective action taken by Alex\'s team in response to a message from an unknown intelligence.")##\n("relationship"<|>"Sam Rivera"<|>"Intelligence"<|>"Sam Rivera is directly involved in the process of learning to communicate with the unknown intelligence."<|>9)##\n("relationship"<|>"Alex"<|>"First Contact"<|>"Alex leads the team that might be making the First Contact with the unknown intelligence."<|>10)##\n("relationship"<|>"Alex"<|>"Humanity\'s Response"<|>"Alex and his team are the key figures in Humanity\'s Response to the unknown intelligence."<|>8)##\n("relationship"<|>"Control"<|>"Intelligence"<|>"The concept of Control is challenged by the Intelligence that writes its own rules."<|>7)<|COMPLETE|>\n#############################\n-Real Data-\n######################\nEntity_types: disease,symptom,cause\nText: atelectasis (summary) | radiology reference article | radiopaedia.org\nx\nrecent edits\nlog in\narticles\nsign up\ncases\ncourses\nquiz\ndonate\nabout\nx\nmenu\nsearch\nadvertisement: radiopaedia is free thanks to our supporters and advertisers.\nbecome a gold supporter and see no third-party ads.\narticles\ncases\ncourses\nlog in\nlog insign up\n{"url":"/signup-modal-props.json?lang=gb"}\narticles\ncases\ncourses\nquiz\nabout\nrecent edits\ngo ad-free\nsearch\natelectasis (summary)\nlast revised by yuranga weerakkody on 30 oct 2021\nedit article\nreport problem with article\nview revision history\ncitation, doi, disclosures and article data\ncitation:\njones j, weerakkody y, worsley c, et al. atelectasis (summary). reference article, radiopaedia.org (accessed on 17 jul 2024) https://doi.org/10.53347/rid-51373\ndoi:\nhttps://doi.org/10.53347/rid-51373\npermalink:\nhttps://radiopaedia.org/articles/51373\nrid:\n51373\narticle created:\n13 feb 2017,\njeremy jones *\ndisclosures:\nat the time the article was created jeremy jones had no recorded disclosures.\nview jeremy jones\'s current disclosures\nlast revised:\n30 oct 2021,\nyuranga weerakkody *\ndisclosures:\nat the time the article was last revised yuranga weerakkody had no recorded disclosures.\nview yuranga weerakkody\'s current disclosures\nrevisions:\n14 times, by\n8 contributors -\nsee full revision history and disclosures\nsystems:\nchest\ntags:\nmedical student, uoe, summary\nthis is a basic article for medical students and other non-radiologistsatelectasis describes small areas of collapsed lung. atelectasis and collapse both describe the same pathophysiology, though atelectasis tends to be used to describe small areas of lung that are not fully expanded, whereas collapse tends to be used to describe larger more confluent areas.\non this page:\narticle:\nreference article\nsummary\nradiographic features\nrelated articles\nimages:\ncases and figures\nimaging differential diagnosis\nreference articlethis is a summary article; read more in our article on atelectasis. summary\nepidemiologyvaries widely depending on the underlying cause\npresentationusually breathlessness, but can be asymptomatic and presentation also depends very much on the underlying cause\npathophysiologycaused by:\nadjacent compressione.g. a lung tumour, dilated aorta, large osteophyte\npassive atelectasiswhen the lung relaxes away from the parietal pleural surface, e.g. pleural effusion, pneumothorax\ndependent atelectasisin the posterior portions of the lung due to patients not fully expanding their lungs while lying for long periods\nrole of imaging\nconfirm atelectasisdifferentiate from air-space opacification\nhelp to determine the cause (may need ct)\ntreatment\nmanagement of the underlying cause\ndeeper breathing if the cause is due to hypoventilation\nadvertisement: supporters see fewer/no ads\nradiographic featureschest x-rayatelectasis is usually seen on chest x-rays as small volume linear shadows, usually peripherally or at the lung bases. the underlying cause (such as a lung tumour or pleural effusion) may also be visible. lobar collapse will have a more typical and appearance based on the lobe involved, whereas atelectasis can be more eccentric in position and appearance.ct chestct allows a more accurate depiction of the involved lung. the dependent lung can be affected by subsegmental collapse just because of lying down (particularly if the patient has been supine for a long period of time, or they are under general anaesthesia). this part of the lung is easier to visualise because it is hidden behind the diaphragm on a frontal chest x-rayct is often helpful to determine the cause or confirm that there is no proximal obstruction, though most cases of atelectasis are diagnosed and managed without ct investigation.\nincoming links\narticles:\nair-bronchogram (summary)\nrelated articles: education: medical student curriculum\nradiology for students\nneuroradiology\nimaging\nct head\nmri brain\nus carotids\nkey findings\nmidline shift\nmass effect\nhydrocephalus\nbrain mass lesion\ncerebral oedema\nintracranial haemorrhage\nextra-axial collection\nconditions\nextradural haemorrhage\nsubdural haemorrhage\nsubarachnoid haemorrhage\nischaemic stroke\nhaemorrhagic stroke\nintracranial tumours\ncerebral abscess\nmultiple sclerosis\nskull fractures\nspinal cord compression\npresentations\nhead injury\nfocal weakness\nsevere headache\nseizures\naltered consciousness\ncauda equina syndrome\ncardiac radiology\nchest radiology\nimaging\nchest x-ray\nct chest\nkey findings\nair-space opacification\natelectasis\nair bronchogram\npneumothorax\npleural fluid\nincreased cardiothoracic ratio\npneumomediastinum\nsurgical emphysema\nconditions\npneumonia\nlobar collapse\npulmonary oedema\nheart failure\nsymptomatic pneumothorax\ntension pneumothorax\npleural effusion\ncopd\nasthma\npulmonary embolism\nlung cancer\nmesothelioma\nbronchiectasis (and cf)\ntuberculosis\npresentations\nbreathlessness\n######################\nOutput:',
    },
]
res = pipe(
    messages,
    # do_sample=False,
    # temperature=1,
    # top_p=0,
    max_new_tokens = 250,
)

print(res[0]["generated_text"][-1])  # Prin

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
c:\Users\mike8\anaconda3\envs\pytorch\Lib\site-packages\transformers\models\llama\modeling_llama.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'role': 'assistant', 'content': '## ("entity"<|>"Atelectasis"<|>"disease"<|>"Atelectasis is a medical condition characterized by small areas of collapsed lung.")##\n## ("entity"<|>"Collapse"<|>"symptom"<|>"Collapse refers to the same pathophysiology as atelectasis, describing small areas of lung that are not fully expanded.")##\n## ("entity"<|>"Atelectasis (summary)"<|>"cause"<|>"Atelectasis is caused by various factors, including adjacent compression, passive atelectasis, and dependent atelectasis.")##\n## ("entity"<|>"Adjacent compression"<|>"cause"<|>"Adjacent compression is a cause of atelectasis, where a lung tumor, dilated aorta, or large osteophyte compresses the lung.")##\n## ("entity"<|>"Passive atelectasis"<|>"cause"<|>"Passive atelectasis occurs when the lung relaxes away from the parietal pleural surface, often due to a pleural effusion or pneumothorax.")##\n## ("entity"<|>"Dependent atelectasis"<|>"cause"<|>"Dependent atelectasis occurs in the posterior portions of'}


In [4]:
len(res[0]["generated_text"][-1]['content'].split(" "))

97

In [26]:
tokenized_chat = pipe.tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
)

In [28]:
outputs = pipe.model.generate(tokenized_chat.cuda(), max_new_tokens=128) 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [30]:
pipe.tokenizer.decode(outputs[0])

'<|im_start|>user\n\n-Goal-\nGiven a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.\n\n-Steps-\n1. Identify all entities. For each identified entity, extract the following information:\n- entity_name: Name of the entity, capitalized\n- entity_type: One of the following types: [disease,symptom,cause]\n- entity_description: Comprehensive description of the entity\'s attributes and activities\nFormat each entity as ("entity"<|><entity_name><|><entity_type><|><entity_description>\n\n2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.\nFor each pair of related entities, extract the following information:\n- source_entity: name of the source entity, as identified in step 1\n- target_entity: name of the target entity, as identified in step 1\n- relationship_descript

In [12]:
import openai
from secret import OPENAI_API_KEY

openai.api_key = OPENAI_API_KEY
openai_res = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

In [15]:
openai_res.choices[0].message

ChatCompletionMessage(content='("entity"<|>"Atelectasis"<|>"disease"<|>"Atelectasis describes small areas of collapsed lung, with different causes and manifestations, often visible on imaging such as chest x-rays and CT scans.")##\n("entity"<|>"Collapsed Lung"<|>"disease"<|>"Collapsed Lung describes the same pathophysiology as atelectasis, with small areas not fully expanded, distinguished from larger confluent areas.")##\n("entity"<|>"Radiology Reference Article"<|>"technology"<|>"Radiology Reference Article provides information on atelectasis and its characteristics for medical professionals and non-radiologists.")##\n("relationship"<|>"Atelectasis"<|>"Collapsed Lung"<|>"Atelectasis and Collapsed Lung have similar pathophysiology, with atelectasis describing smaller areas of collapse compared to the larger confluent areas in Collapsed Lung."<|>8)##\n("relationship"<|>"Atelectasis"<|>"Radiology Reference Article"<|>"The Radiology Reference Article provides detailed information on atel